# Deelmodel Steenbergerloop

In [1]:
import hydrotools
from pathlib import Path
import geopandas as gpd
from delft3dfmpy import DFlowFMModel, DFlowFMWriter

In [2]:
hydamo = hydrotools.load_model(Path(r"./hydamo_model/boezemmodel_v4.pickle"))
shp_file = r".\shp\steenbergerloop.shp"
geometry = gpd.read_file(shp_file).loc[0]['geometry']
hydamo = hydrotools.filter_model(hydamo, geometry=geometry)
hydrotools.export_shapes(hydamo, path=Path(r"./hydamo_shp/steenbergerloop"))

## Converteren naar DFM

### Aanmaken dfm-klasse

In [3]:
dfmmodel = DFlowFMModel()

### Inlezen kunstwerken

In [4]:
dfmmodel.structures.io.weirs_from_hydamo(hydamo.weirs,
                                         yz_profiles=hydamo.crosssections,
                                         parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.culverts_from_hydamo(hydamo.culverts,
                                            hydamo.afsluitmiddel)

dfmmodel.structures.io.bridges_from_hydamo(hydamo.bridges,
                                           yz_profiles=hydamo.crosssections,
                                           parametrised_profiles=hydamo.parametrised_profiles)

dfmmodel.structures.io.orifices_from_hydamo(hydamo.orifices)

dfmmodel.structures.io.pumps_from_hydamo(pompen=hydamo.pumps,
                                         sturing=hydamo.sturing,
                                         gemalen=hydamo.gemalen)

### Aanmaken 1d netwerk

In [5]:
dfmmodel.network.set_branches(hydamo.branches)
dfmmodel.network.generate_1dnetwork(one_d_mesh_distance=100.0, seperate_structures=True)

### Toevoegen cross-sections

In [6]:
dfmmodel.crosssections.io.from_hydamo(
    dwarsprofielen=hydamo.crosssections,
    parametrised=hydamo.parametrised_profiles,
    branches=hydamo.branches
)

print(f"{len(dfmmodel.crosssections.get_branches_without_crosssection())} branches are still missing a cross section.")
print(f"{len(dfmmodel.crosssections.get_structures_without_crosssection())} structures are still missing a cross section.")

No parametrised crossections available for branches.
867 branches are still missing a cross section.
0 structures are still missing a cross section.


### Wegschrijven model

In [7]:
dfmmodel.mdu_parameters["refdate"] = 20000101
dfmmodel.mdu_parameters["tstart"] = 0.0 * 3600
dfmmodel.mdu_parameters["tstop"] = 144.0 * 1 * 3600
dfmmodel.mdu_parameters["hisinterval"] = "120. 0. 0."
dfmmodel.mdu_parameters["cflmax"] = 0.7

dimr_path = r"dummypath\run_dimr.bat"
dfmmodel.dimr_path = dimr_path
fm_writer = DFlowFMWriter(dfmmodel, output_dir="steenbergerloop", name="steenbergerloop")

fm_writer.objects_to_ldb()
fm_writer.write_all()

if Path(r".\steenbergerloop\fm\steenbergerloop.mdu").exists():
    print("Model is weggeschreven")
else:
    print("Er is geen model geschreven. Waarschijnlijk is iets fout gegaan")

Model is weggeschreven
